# Curso Análisis y limpieza de datos

**Profesor responsable:** Jorge Alexis Castillo Sepúlveda, PhD, MSc & Math. Eng.

## Capitulo 1: Limpieza de datos

### Clase 1: Obtención , organización y manejo de datos usando la librería Pandas de Python

En esta clase veremos como obtener datos y cierto manejo básico en la librería Pandas de Python de modo de poder inferir que tipo de datos estamos trabajando, usando más de un caso de estudio.

### Dataset clásicos

En el ámbito de la *ciencia de datos*, es usual hablar de *datsets clásicos*. Estos datasets generalmente vienen incluídos en paquetes y han servido como ejemplos prácticos a la hora de aprender métodos en esta disciplina, a lo largo de varias generaciones de estudiantes. Nosotros no seremos la excepción y nos servirán para explorar los datos y aprender a usar los comandos básicos.



### Datasets no clásicos

Por otra parte, no siempre tendremos a nuestra disposición datasets clásicos. Es más, en las aplicaciones de verdad, ya sea en la academia o en la industria, tendremos que acostumbrarnos a usar datasets "sucios", o que provienen de fuentes extrañas, o que no estén en un formato deseado. El primer paso de un *data scientist* es lidiar con ese tipo de datasets y es bueno que empecemos a acostumbrarnos a trabajar con ellos. 

### Importando un csv (comma separated values) desde una carpeta específica 

In [77]:
import os
os.chdir('C:/Users/aleph/Dropbox/Docencia posgrado/Analisis de Datos  MDS19/datasets')

"""
Dentro del paréntesis deben poner la URL de la carpeta donde guardaron el archivo IRIS.csv
"""

'\nDentro del paréntesis deben poner la URL de la carpeta donde guardaron el archivo IRIS.csv\n'

In [78]:
#importando la libreria pandas
import pandas as pd # despues del "as" puedes poner LO QUE QUIERAS, pero por convención, se usa pd

In [86]:
data=pd.read_csv('IRIS.csv', sep=',')

#generalmente se usa df, y de nuevo, tu le puedes poner el nombre que quieras ;)

In [91]:
# viendo el encabezado de la data para ver como es

data.head(170000) #dentro del parentesis puedes poner el numero entero que quieras 

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
5,5.4,3.9,1.7,0.4,Iris-setosa
6,4.6,3.4,1.4,0.3,Iris-setosa
7,5.0,3.4,1.5,0.2,Iris-setosa
8,4.4,2.9,1.4,0.2,Iris-setosa
9,4.9,3.1,1.5,0.1,Iris-setosa


In [96]:
aux=data.head(100)

In [97]:
aux

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
5,5.4,3.9,1.7,0.4,Iris-setosa
6,4.6,3.4,1.4,0.3,Iris-setosa
7,5.0,3.4,1.5,0.2,Iris-setosa
8,4.4,2.9,1.4,0.2,Iris-setosa
9,4.9,3.1,1.5,0.1,Iris-setosa


In [98]:
# ¿que tipos de datos tiene nuestro dataframe?
data.dtypes

sepal_length    float64
sepal_width     float64
petal_length    float64
petal_width     float64
species          object
dtype: object

In [99]:
# ¿como se mueven las variables continuas?
data.describe()

,sepal_length,sepal_width,petal_length,petal_width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667
std,0.828066,0.433594,1.764420,0.763161
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [100]:
# ¿y qué pasa con la columna "species"?

#veamos las categorías que contiene:
data.species.unique()

# el método unique sirve para  mostrar las posibilidades únicas de cada columna. Su equivalente en R es unique(...)

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

#### Observación: Python es un lenguaje muy intuitivo y es posible usar los "métodos" para denominar a las columnas. Pero para que esto sea posible, el string o label de la columna debe ser una "unidad" completa o monomio. Si tiene un nombre que exceda el "monomio", se recomienda usar el guión bajo para generar el monomio.

#### Suponga que un investigador a cargo del área donde usted trabaja le dice que sólo nos interesa analizar la especie iris versicolor, ya que esa especie tiene recomendaciones para problemas hepáticos.

Vamos a crear un dataframe donde solo esté esa especie.


In [104]:
#En pandas, para filtrar se debe usar el comando lógico "==" 

df_aux=data[data.species=='Iris-virginica']

In [105]:
df_aux

,sepal_length,sepal_width,petal_length,petal_width,species
100,6.3,3.3,6.0,2.5,Iris-virginica
101,5.8,2.7,5.1,1.9,Iris-virginica
102,7.1,3.0,5.9,2.1,Iris-virginica
103,6.3,2.9,5.6,1.8,Iris-virginica
104,6.5,3.0,5.8,2.2,Iris-virginica
105,7.6,3.0,6.6,2.1,Iris-virginica
106,4.9,2.5,4.5,1.7,Iris-virginica
107,7.3,2.9,6.3,1.8,Iris-virginica
108,6.7,2.5,5.8,1.8,Iris-virginica
109,7.2,3.6,6.1,2.5,Iris-virginica


In [106]:
#Describiendo los datos

df_aux.describe()

,sepal_length,sepal_width,petal_length,petal_width
count,50.00000,50.000000,50.000000,50.00000
mean,6.58800,2.974000,5.552000,2.02600
std,0.63588,0.322497,0.551895,0.27465
min,4.90000,2.200000,4.500000,1.40000
25%,6.22500,2.800000,5.100000,1.80000
50%,6.50000,3.000000,5.550000,2.00000
75%,6.90000,3.175000,5.875000,2.30000
max,7.90000,3.800000,6.900000,2.50000


#### Ejercicio: inferencias sobre los datos:

- Si un colega que está trabajando con los mismos datos, le dice que vio en el dataset una Iris-virginica con sétalo de largo 4.845 . ¿Usted le creería?
- Si alguien dice que ha visto Iris-virginica con pétalos de largo 7, ¿qué le diría?


#### Para indagar:

- Lea la documentación correspondiente a los métodos **describe** y **unique** de la librería pandas y aplique algo distinto al paréntesis vacío. 

### Importando un csv desde una url


In [107]:
#### importando desde una URL

data2=pd.read_csv('https://bit.ly/2ZNXHTQ')


In [108]:
data2.head(10)

,año,mes,dia,cantidad,venta
0,2019,2,1,125.8,79647.1
1,2019,2,10,255.0,164550.1
2,2019,2,1,255.0,163424.3
3,2018,2,1,391.0,242205.6
4,2018,7,1,181.9,105966.9
5,2018,2,10,470.9,291720.0
6,2019,2,10,149.6,96317.0
7,2018,2,1,214.2,133337.1
8,2017,2,3,723.0,346609.2
9,2018,7,10,467.5,272746.5


In [109]:
data2.describe()

,año,mes,dia,cantidad,venta
count,1616.000000,1616.000000,1616.000000,1616.000000,1.616000e+03
mean,2017.665842,6.023515,15.750000,493.911696,2.690113e+05
std,0.665633,3.552796,8.763667,846.233046,3.551489e+05
min,2017.000000,1.000000,1.000000,51.000000,3.303950e+04
25%,2017.000000,3.000000,8.000000,263.875000,1.594148e+05
50%,2018.000000,6.000000,16.000000,372.300000,2.186399e+05
75%,2018.000000,9.000000,23.000000,523.950000,2.963082e+05
max,2019.000000,12.000000,31.000000,14189.900000,5.966692e+06


In [110]:
data2.dtypes

año           int64
mes           int64
dia           int64
cantidad    float64
venta       float64
dtype: object

In [113]:
#cada columna representa una serie
type(data2.dia)

pandas.core.series.Series

Parece ser que tenemos un datset que es una serie de tiempo, en donde las columnas que tienen que ver con la fecha, están separadas y no se ve un orden aparente. Hay otras dos coliumnas que corresponden a cantidad y venta, es decir, estamos hablando de un dataset que proviene directamente de la industria.

#### Creando la columna de fechas

Lo que hay que hacer, es crear una columuna uniendo lo que se ve en la columna de año, en la columna de mes y en la columna de día. ¿Cómo lo hacemos? Hay más de una manera. 

### Método 1: Concatenar

In [115]:
#esto lanzará un error

data2['fecha']=str(data2.dia)+ '-' + str(data2.mes)+'-'+str(data2.año)
data2['fecha']

0       0        1\n1       10\n2        1\n3        1...
1       0        1\n1       10\n2        1\n3        1...
2       0        1\n1       10\n2        1\n3        1...
3       0        1\n1       10\n2        1\n3        1...
4       0        1\n1       10\n2        1\n3        1...
5       0        1\n1       10\n2        1\n3        1...
6       0        1\n1       10\n2        1\n3        1...
7       0        1\n1       10\n2        1\n3        1...
8       0        1\n1       10\n2        1\n3        1...
9       0        1\n1       10\n2        1\n3        1...
10      0        1\n1       10\n2        1\n3        1...
11      0        1\n1       10\n2        1\n3        1...
12      0        1\n1       10\n2        1\n3        1...
13      0        1\n1       10\n2        1\n3        1...
14      0        1\n1       10\n2        1\n3        1...
15      0        1\n1       10\n2        1\n3        1...
16      0        1\n1       10\n2        1\n3        1...
17      0     

**A tener en cuenta** : Tener errores no nos hace menos data scientists ni nada por el estilo. No existe ser humano que no haya tenido errores en programación. Los errores sirven para aprender, es la gracia de la investigación y la ciencia en general. Quien busque la perfección debe replantearse. 

*¿Donde está el error?* En que se quiere concatenar datos que no son string (excepto por el guión "-")

**La solución es usar el método map, cambiando el tipo de dato a string**. Esto es porque cada columna es una Serie (Series) y map actúa sobre cada elemento de una serie.

In [116]:
data2['fecha']=data2.dia.map(str) + '-' + data2.mes.map(str) + '-' + data2.año.map(str)

In [117]:
data2['fecha']

0         1-2-2019
1        10-2-2019
2         1-2-2019
3         1-2-2018
4         1-7-2018
5        10-2-2018
6        10-2-2019
7         1-2-2018
8         3-2-2017
9        10-7-2018
10       10-2-2018
11        1-3-2017
12        1-7-2018
13        4-2-2017
14       1-10-2018
15        6-2-2017
16        1-8-2017
17       10-1-2019
18       12-2-2019
19       10-1-2019
20       10-5-2018
21       10-3-2017
22        7-2-2017
23       10-7-2018
24       13-6-2017
25       11-2-2019
26        1-3-2017
27      10-10-2018
28       11-1-2019
29        5-2-2017
           ...    
1586      7-8-2018
1587     9-12-2017
1588     23-2-2017
1589     9-12-2017
1590     20-2-2017
1591     8-12-2017
1592     22-2-2017
1593      8-8-2018
1594      9-8-2018
1595     24-2-2017
1596      9-8-2018
1597     24-2-2017
1598     21-2-2017
1599     22-2-2017
1600     25-2-2017
1601     25-2-2017
1602     26-2-2017
1603     28-2-2017
1604     28-2-2017
1605     27-2-2017
1606     26-2-2017
1607     27-

La idea entonces es ordenar los datos por fecha. Veamos que pasa si ordeno solo la columna de fechas recién creada.


In [118]:
data2

,año,mes,dia,cantidad,venta,fecha
0,2019,2,1,125.8,79647.1,1-2-2019
1,2019,2,10,255.0,164550.1,10-2-2019
2,2019,2,1,255.0,163424.3,1-2-2019
3,2018,2,1,391.0,242205.6,1-2-2018
4,2018,7,1,181.9,105966.9,1-7-2018
5,2018,2,10,470.9,291720.0,10-2-2018
6,2019,2,10,149.6,96317.0,10-2-2019
7,2018,2,1,214.2,133337.1,1-2-2018
8,2017,2,3,723.0,346609.2,3-2-2017
9,2018,7,10,467.5,272746.5,10-7-2018


In [119]:
data2.sort_values(by=['fecha']) 

,año,mes,dia,cantidad,venta,fecha
207,2017,10,1,1269.0,692805.6,1-10-2017
189,2017,10,1,874.5,479674.8,1-10-2017
14,2018,10,1,372.3,213961.8,1-10-2018
45,2018,10,1,425.0,262874.3,1-10-2018
258,2017,11,1,654.0,349706.4,1-11-2017
200,2017,11,1,655.5,396627.6,1-11-2017
157,2018,11,1,166.6,108290.0,1-11-2018
117,2018,11,1,244.8,154700.0,1-11-2018
154,2017,12,1,880.5,489667.2,1-12-2017
119,2017,12,1,508.5,281546.4,1-12-2017


**¿qué pasó? ¿es correcto el orden?**

####  Debemos transformar el string a fecha usando pd.to_datetime

In [121]:
data2['fecha']=pd.to_datetime(data2['fecha'])

In [122]:
data2.sort_values(by=['fecha'], inplace=True) # se puede usar con inplace=True para no tener que hacer x=x(condicion nueva)

In [123]:
data2

,año,mes,dia,cantidad,venta,fecha
655,2017,2,1,937.5,483481.2,2017-01-02
676,2017,2,1,352.5,180204.0,2017-01-02
26,2017,3,1,1107.0,577962.0,2017-01-03
11,2017,3,1,673.5,348318.0,2017-01-03
69,2017,4,1,280.5,154766.4,2017-01-04
43,2017,4,1,408.0,226039.2,2017-01-04
1286,2017,6,1,280.5,173910.0,2017-01-06
1310,2017,6,1,513.0,318060.0,2017-01-06
169,2017,7,1,568.5,338064.0,2017-01-07
124,2017,7,1,631.5,374394.0,2017-01-07


In [124]:
# Sólo nos interesa la fecha, la venta y la cantidad, por lo tanto, seleccionamos las columans de interés:

data2=data2[['fecha','cantidad','venta']]
data2


,fecha,cantidad,venta
655,2017-01-02,937.5,483481.2
676,2017-01-02,352.5,180204.0
26,2017-01-03,1107.0,577962.0
11,2017-01-03,673.5,348318.0
69,2017-01-04,280.5,154766.4
43,2017-01-04,408.0,226039.2
1286,2017-01-06,280.5,173910.0
1310,2017-01-06,513.0,318060.0
169,2017-01-07,568.5,338064.0
124,2017-01-07,631.5,374394.0


In [126]:
""""
Por último, observamos que los indices del nuevo dataframe están desordenados. 
Para el análisis de datos, en ocasiones es MUY IMPORTANTE que los índices estén ordenados
Para eso usamos el método reset_index. Debemos usar drop si no queremos que el índice antiguo 
se transforme en una columna nueva
"""""
data2.reset_index(inplace=True, drop=True)
data2

,fecha,cantidad,venta
0,2017-01-02,937.5,483481.2
1,2017-01-02,352.5,180204.0
2,2017-01-03,1107.0,577962.0
3,2017-01-03,673.5,348318.0
4,2017-01-04,280.5,154766.4
5,2017-01-04,408.0,226039.2
6,2017-01-06,280.5,173910.0
7,2017-01-06,513.0,318060.0
8,2017-01-07,568.5,338064.0
9,2017-01-07,631.5,374394.0


### Método 2: Usando el método  apply, format y el cálculo lambda 

- El método **apply** sirve para aplicar una función a toda una columna o toda una fila en un determinado dataframe en pandas.
- El formateo (format) hace que cada expresion vacía se llene con lo que está dentro del parentesis '{}'. Por lo tanto es una manera *simple* de pasar a string.   
- EL **cálculo lambda** proviene de la lógica matemática para definir el concepto de función y abordar el problema de la recursividad. En python se usa para definir funciones de una manera simple y directa usando la siguiente lógica:

<h3><center>lambda variable: expresion(variable)</center></h3>


Por lo tanto, este método, que es más sotisificado que el anterior, consiste en aplicar a toda la columna un formateo de string usando una función definida por el cálculo lambda. Luego pasamos a formato de fecha y reseteamos el índice. 

In [127]:
data3=pd.read_csv('https://bit.ly/2ZNXHTQ')

#en una sola línea:
data3['fecha']=data3[['dia','mes','año']].apply(lambda x : '{}-{}-{}'.format(x[0],x[1],x[2]), axis=1)

###

data3['fecha']=pd.to_datetime(data3['fecha'])
data3=data3[['fecha','cantidad','venta']]
data3.sort_values('fecha', inplace=True)
data3.reset_index(inplace=True, drop=True)
data3

,fecha,cantidad,venta
0,2017-01-02,937.5,483481.2
1,2017-01-02,352.5,180204.0
2,2017-01-03,1107.0,577962.0
3,2017-01-03,673.5,348318.0
4,2017-01-04,280.5,154766.4
5,2017-01-04,408.0,226039.2
6,2017-01-06,280.5,173910.0
7,2017-01-06,513.0,318060.0
8,2017-01-07,568.5,338064.0
9,2017-01-07,631.5,374394.0


### Ejercicios

- Calcular una columna de precio
- El cliente necesita las salidas en el formato YY-MM-DD. Cree otro dataset con las fechas en ese formato usando cualquier de los dos métodos (de preferencia el 2)
- Suponga que un data scientist senior de su empresa le pide analizar **un año de datos** con fecha máxima el mes julio 2019, ya que se debe evaluar las ventas del ultimo año y en producción aun no se cierra el mes de agosto 2019. Cree un dataframe auxiliar con esta consideración.
- ¿Por qué hay cree usted que hay dos datos por cada fecha?

**Tarea**

- Cree un nuevo dataset en donde cada fecha debe aparecer una sola vez.
- Calcule el precio promedio del último año cerrado
- ¿En cuántos días las transacciones estuvieron por sobre los 650 pesos? 

In [130]:
data3['precio']=data3.venta/data3.cantidad
data3

,fecha,cantidad,venta,precio
0,2017-01-02,937.5,483481.2,515.713280
1,2017-01-02,352.5,180204.0,511.217021
2,2017-01-03,1107.0,577962.0,522.097561
3,2017-01-03,673.5,348318.0,517.175947
4,2017-01-04,280.5,154766.4,551.751872
5,2017-01-04,408.0,226039.2,554.017647
6,2017-01-06,280.5,173910.0,620.000000
7,2017-01-06,513.0,318060.0,620.000000
8,2017-01-07,568.5,338064.0,594.659631
9,2017-01-07,631.5,374394.0,592.864608


In [131]:
data3.precio.describe()

count    1616.000000
mean      584.013880
std        51.197193
min       398.356334
25%       546.141427
50%       595.527692
75%       623.200000
max       650.000000
Name: precio, dtype: float64